# Optimisation de la prédiction du risque de blessure pour athlètes de haut niveau

**Questions pour Nelly Pustelnik**

Nous avons trois grosses questions :

1. La question du modèle : nous avons choisi d’utiliser une régression logistique pour modéliser la probabilité de blessure des athlètes. Cependant, ce modèle n’est peut-être pas le plus approprié pour notre problématique (du moins tel que nous l’avons formulée). De plus, il est possible que les variables explicatives disponibles ne soient pas suffisamment informatives pour prédire la survenue d’une blessure, phénomène qui peut dépendre de facteurs non observés dans nos données. Ainsi, bien que nos algorithmes convergent correctement, ils aboutissent à des performances prédictives très très faibles (à peine plus de 50%).

2. Comme indiqué dans le Notebook, les données initiales sont fortement déséquilibrées : moins de 0,01 % des observations correspondent à des blessures.
Pour pallier ce déséquilibre, nous avons adopté une stratégie de sous-échantillonnage :
  * Nous conservons toutes les observations de blessure
  * Nous tirons aléatoirement un sous-échantillon d’observations sans blessure afin d’obtenir un jeu de données contenant environ 20 % de blessés.
  * Nous considérons ensuite ce sous-échantillon comme notre nouvel échantillon de travail pour l’ensemble du projet. Nous utilisons des poids dans la fonction de perte pour contre-carrer le déséquilibre restant.
Notre question : ce choix méthodologique est-il acceptable?

3. L'algorithme Chambolle-Pock. Nous rencontrons plusieurs difficultés avec l’implémentation de cet algorithme :
* Nous ne parvenons pas à déterminer le prox de notre fonction $h$ telle que définie ci-dessous. La bibliothèque prox-repository propose un prox pour la fonction $\log(1+\exp(x))$, mais il ne semble pas directement applicable à notre cas, la fonction $h$ n’étant pas séparable en les coordonnées de $\theta$.
* En conséquence, nous avons pour l’instant remplacé le calcul du prox de $h$ par une descente de gradient.
* Enfin, nous nous interrogeons sur le choix des paramètres $\tau$ et $\sigma$ dans l'algorithme de Chambolle-Pock : à part la contrainte $\tau \sigma \leq 1$ avons nous d'autres conditions qui garantissent leur validité et leur stabilité dans notre contexte ?

**Objectif :** prédire le risque de blessure d'un coureur sur un cycle d'entraînement.

Les données : Notre base de données contient des données d'entraînement de 74 athlètes de haut niveau sur une période de 7 ans, avec des données qui sont au format hebdomadaire. La variable cible (blessure ou non) y est renseignée, ainsi que différentes covariables telles que :
* Nombre de kilomètres
* Intensité de l'entraînement
* Type d'entraînement
* Qualité perçue de la récupération
* etc.

[Source des données](https://dataverse.nl/dataset.xhtml?persistentId=doi:10.34894/UWU9PV)

## Modélisation du problème
### Regression logistique
**A voir** : peut-être qu'on veut rendre plus synthétique toute cette partie modélisation? Jsp à quel point ce document doit être court ou pas...

On se donne un modèle de régression logistique:
$$ P(Y = 1 \mid X) = \eta_{\theta}(X) = \frac{1}{1 + e^{-(X^\top \theta)}}. $$

où le paramètre $X$ représente les données d'entraînement collectées sur la période de suivi sur lesquelles on ajoute une colonne de 1 pour capter une ordonnée à l'origine, et le paramètre $Y
 \in \{0,1\}$ représente le fait de se blesser sur la période ($Y=1$) ou non $(Y=0)$.

 On se donne une règle de décision bayésienne à partir d'un vecteur d'observation $X$ (associé à une nouvelle semaine d'observation):
$$
\begin{array}{c l}
\text{Prediction =  1} & \text{si } \eta_{\theta}(X) \geq \frac{1}{2}, \\
\text{Prediction = 0} & \text{sinon.}
\end{array}
$$

Nous cherchons à estimer le vecteur $\theta$ qui encode la relation de chacune des données au risque de blessure.

Dans une approche statistique, on peut estimer ce paramètre en minimisant l'opposé de la log-vraisemblance conditionnelle, ce qui donne le problème d'optimisation suivant:
$$
\hat{\theta} = \arg\min_{\theta \in \mathbb{R}^p} -\sum_{i=1}^n Y_i \log (\eta_{\theta}(X_i)) + (1-Y_i) \log (1-\eta_{\theta}(X_i)).
$$


Mais quelques problèmes émergent avec cette méthode d'estimation:
* Nous avons dans notre base de données  une prévalence nette des non-blessures sur les blessures: l'aléa entre prédire 1 et 0 n'est donc pas le même.
* La parcimonie de notre modèle n'est pas garantie: on peut estimer des coefficients $\theta_{j}$ en réalité pas très pertinents pour la prédiction du modèle.


### Modélisation
Pour pallier aux problèmes inhérents au modèle présenté précédemment, nous procédons de la manière suivante:
* Ajouter des pondérations $\omega_{0}$ et $\omega_{1}$ aux différentes classes avec nécessairement $\omega_{1} > \omega_{0}$ (avec par ex: $\omega_{1} = \frac{n}{2n_{1}}$ et $\omega_{0} = \frac{n}{2n_{0}}$ qu'on peut par la suite renormaliser avec la règle $\hat{\omega_{i}} = \frac{\omega_{i}}{\omega_{0} + \omega_{1}})$

* Ajouter une contrainte générale sur les $\theta_{j}$ de la forme $\sum_{j=1}^p |\theta_{j}| \leq \alpha  $ (cela va permettre de forcer certains coefficients à 0 pour respecter la contrainte et de ne garder in fine qu'un nombre réduit de paramètres

On obtient alors le nouveau problème d'optimisation:
$$
\hat{\theta} =
\arg\min_{\theta \in \mathbb{R}^p} -\sum_{i=1}^n \omega_{1} Y_i \log (\eta(X_i)) + \omega_{0}(1-Y_i) \log (1-\eta(X_i)) , \quad \text{s.c. } \sum_{j=1}^p |\theta_j| \leq \alpha
$$

On peut alors construire le problème dual :
\begin{align*}
\max_{\lambda \ge 0} \; \min_{\theta \in \mathbb{R}^p}
\mathcal{L}(\theta, \lambda) &=
\max_{\lambda \ge 0} \; \min_{\theta \in \mathbb{R}^p}
\Bigg[
- \sum_{i=1}^n \left[ \omega_1 Y_i \log (\eta(X_i)) + \omega_0 (1-Y_i) \log (1-\eta(X_i)) \right]
+ \lambda \|\theta\|_1 - \lambda \alpha
\Bigg] \\
&=
\max_{\lambda \ge 0} \; \min_{\theta \in \mathbb{R}^p}
\Bigg[
- \sum_{i=1}^n \left[ \omega_1 Y_i \log (\eta(X_i)) + \omega_0 (1-Y_i) \log (1-\eta(X_i)) \right]
+ \lambda \|\theta\|_1
\Bigg]
\end{align*}

Nous allons nous concentrer ici sur la résolution de la forme pénalisée. Notre problème d'optimisation final est donc le suivant:
$$
\min_{\theta \in \mathbb{R}^p}
\Bigg[
- \sum_{i=1}^n \left[ \omega_1 Y_i \log (\eta(X_i)) + \omega_0 (1-Y_i) \log (1-\eta(X_i)) \right]
+ \lambda \|\theta\|_1
\Bigg]
$$
Que nous pourrions également formuler:
$$ \min_{\theta \in \mathbb{R}^p} h(\theta) + f(\theta)
$$

Avec les fonctions $h$ et $f$ définies par:
$$ \begin{align}
& h : \quad && \mathbb{R}^p \to \mathbb{R} \\
& && \theta \mapsto - \sum_{i=1}^n [\omega_1 Y_i \text{log}(\eta_\theta(X_i)) + \omega_0(1-Y_i) \text{log}(1-\eta_\theta(X_i))] \\
& f : \quad && \mathbb{R}^p \to \mathbb{R} \\
& && \theta \mapsto \lambda \| \theta \|_1
\end{align}
$$





###Hypothèse sous-jacente (un peu forte):

On considère qu'il n'y a pas de différences intrinsèques entre coureurs (les effets marginaux de chaque variable sont identiques et les différences physiologiques de chaque coureur sont négligeables dans la modélisation).


In [ ]:
# Préambule

import pandas as pd
import numpy as np
import numpy.linalg as npl
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

try:
    from tqdm.auto import tqdm
except ModuleNotFoundError:
    # No progress bar if tqdm is not installed
    def tqdm(*args, **kwargs):
        return args[0] # First argument is the iterable

In [ ]:
pip install proxop

## Prise en main des données

###Description du jeu de données

Avant de se lancer dans l'estimation, il faut comprendre comment est organisé notre base de données. Si l'on simplifie en se concentrant seulement sur le tableau qui constitue l'approche par semaine, nous obtenons la visualisation suivante:

In [ ]:
df2 = pd.read_csv("/content/week_approach_maskedID_timeseries.csv")
#df2 = pd.read_csv("week_approach_maskedID_timeseries.csv")
runners = {aid: group for aid, group in df2.groupby("Athlete ID")}
runner70_df2 = runners[70]
runner70_df2.head(10)



Ici, on se concentre seulement sur 10 semaines d'observations du 70ème athlète. On remarque que l'on a un jeu de variables "courantes" et un jeu de variables "retardées". Les variables retardées sont par exemple: "rel total kms week 0_1", "rel rotal kms week 0_2" et "rel total kms week 1_2". Celles-ci indiquent la variation de la charge d'entraînement entre deux semaines : la variation entre la semaine précédent l'observation et l'observation, la variation entre la semaine -2 précédent l'observation et l'observation ainsi que la variation entre les semaines -2 et -1 avant observation. Ces variables sont à interpréter en terme de pourcentage (1.05 équivaut +5% de charge d'entraînement). De plus, on observe des variables à extension: par exemple : nr sessions, nr sessions.1 et nr sessions.2 qui suivent en réalité la même logique (on comptabilise le nombre de session la semaine courante, la semaine -1 et la semaine-2). C'est donc un jeu de données très riche mais dont les lignes et les colonnes sont fortement corrélées.

### Gestion de la base de données

On se rend vite compte que notre modélisation est en fait mal posée: sur environ 43000 données on a seulement 575 occurrences de blessures. On ne pourra jamais échantillonner sur la totalité des données (c'est trop lourd) et lorsqu'on cherchera à sous-échantilloner il y aura très peu de chances d'obtenir des occurrences de blessures. On se propose de sous-échantilloner en gardant toutes les occurrences de blessures: pour s'assurer d'avoir quand même un échantillon de taille acceptable, on se propose de faire en sorte que la proportion de blessés représente 20% de l'échantillon. On pourra alors suivre notre raisonnement initial et adapter les pondérations $w_0$ et $w_1$ correspondantes.




In [ ]:
# proportion cible
pi_ech_target = 0.2


N_pop = len(df2)
N1 = df2['injury'].sum()
N0 = N_pop - N1
pi_pop = N1 / N_pop

# nombre total dans l'échantillon pour atteindre cette proportion
N_ech = int(N1 / pi_ech_target)
N0_ech = N_ech - N1

# sous-échantillonnage
df1 = df2[df2['injury'] == 1]
df0 = df2[df2['injury'] == 0].sample(n=N0_ech, random_state=42)

# concaténer les deux
df_ech = pd.concat([df1, df0]).sample(frac=1, random_state=42)  # mélanger

# nouvelle proportion
pi_ech = df_ech['injury'].mean()

#nouveaux poids
w1 = len(df_ech)/(2*len(df1))
w0 = len(df_ech)/(2*len(df0))

print(f"Taille de l'échantillon: {len(df_ech)}")
print(f"w1 = {w1:.3f}, w0 = {w0:.3f}")

# Etude des fonctions $h$ et $f$

## La fonction $h$
Les calculs suivants nous permettent d'obtenir le gradient de la fonction $h$ de notre problème telle que définie ci-dessus.

$$h(\theta) = - \sum_{i=1}^n [\omega_1 Y_i \text{log}(\eta_\theta(X_i)) + \omega_0(1-Y_i) \text{log}(1-\eta_\theta(X_i))] $$

La dérivée en $\theta$ de $\eta_{\theta}(x)$ nous donne :
$$
\nabla_{\theta} \eta_\theta(x) = \eta_\theta(x)(1 - \eta_\theta(x))x
$$
D'où :
* $$
\nabla_{\theta} \big[\text{log}(\eta_\theta(x))\big] = (1 - \eta_\theta(x))x
$$
* $$
\nabla_{\theta} \big[\text{log}(1-\eta_\theta(x))\big] = - \eta_\theta(x)x
$$

On obtient donc :
$$
\begin{align}
\nabla h (\theta) & = - \sum_{i = 1}^n \big[ \omega_1 Y_i (1 - \eta_\theta(X_i))X_i  -\omega_0 (1-Y_i)\eta_\theta(X_i)X_i \big] \\
& = \sum_{i = 1}^n \big[ \omega_0 (1-Y_i)\eta_\theta(X_i) - \omega_1 Y_i (1 - \eta_\theta(X_i)) \big]X_i
\end{align}
$$

###

Nous pouvons à présent définir les fonctions suivantes :

In [ ]:
def eta(theta, x):
    return 1/(1+np.exp(-x@theta))

def h(theta, x, y, w0, w1):
    n = eta(theta, x)
    return -np.sum(w1 * y * np.log(n) + w0 * (1-y) * np.log(1-n))

def grad_h(theta, x, y, w0, w1):
    n = eta(theta, x)
    coeff = w0*(1-y)*n - w1*y*(1-n)
    return x.T@coeff

### Etude de la nature $\beta$-Lipschitz du gradient de $h$

L'algorithme FISTA que nous allons implémenter contient un paramètre de pas, $\gamma$, dont la valeur optimale est donnée pour $\gamma \in (0,2/\beta) $ où $\beta$ est la constante de Lipschitz pour le gradient de $h$.

Après calcul, on trouve la valeur de $\beta$ suivante:
$$
\beta = \frac{\max(\omega_0, \omega_1)}{4} \lambda_{max}\left(\sum^n_{i=1} X_iX_i^T \right) = \frac{\max(\omega_0, \omega_1)}{4} \lambda_{max} \left( X^TX  \right)
$$
 où $\lambda_{max}(A)$ désigne la plus grande valeur propre de $A$.

#### Premier calcul pour $\beta$

In [ ]:
X = df_ech.drop(columns=["injury"])
eigvals = np.linalg.eigvalsh(X.T @ X)
lambda_max = eigvals[-1]

#valeur de beta
beta = (max(w0, w1)/4)*lambda_max

print(f"Pas optimal calculé pour FISTA : {2/beta}")

Avec le $\beta$ donné, on obtient un pas optimal presque incalculable (de l'ordre de $10^{-17}$ !). Une méthode proposée est de standardiser (recentrer et réduire les variables) pour ne pas faire exploser le calcul de $\lambda_{max}$.

#### Standardisation des données et recalcul de $\beta$


In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

eigvals = np.linalg.eigvalsh(X_scaled.T @ X_scaled)
lambda_bis = eigvals[-1]
beta_b = (max(w0, w1)/4)*lambda_bis
print(f"Pas optimal calculé pour FISTA : {2/beta_b}") # C'est mieux!


## La fonction $f$
Nous définissons notre $f$ ainsi que sa fonction gradient, qui retourne un élément de la sous-différentielle de $f$, et son prox.


In [ ]:
def f(theta, lambd):
  return lambd * np.sum(np.abs(theta))

def grad_f(theta, lambd):
  return lambd * np.sign(theta)

def prox_f(wx, gamma):
  return np.sign(wx) * np.maximum(np.abs(wx) - gamma, 0)

Nous pouvons à présent appliquer l'algorithme FISTA avec le pas $\gamma$ donné par les bornes optimales.

# Préparation du terrain

On sépare nos données en données train et test, en gardant le format standardisé pour $X$, on veillera à ajouter une colonne de 1 dans les variables "x_training" et "x_validation".


In [ ]:
Y = df_ech["injury"]
x_training, x_validation, y_training, y_validation = train_test_split(
    X_scaled, Y, test_size=0.25, random_state=42) # Défini quatre nouveaux ensembles qui sont nos données pour l'entrainement et pour le test.
    # on laisse 1/4 des données de côté pour le test
    # random_state pour la reproductibilité

N = x_validation.shape[0]
x_validation_int = np.hstack([x_validation, np.ones((N, 1))])
N = x_training.shape[0]
x_training_int = np.hstack([x_training, np.ones((N, 1))])


Fonctions (adaptées et récupérées du TP) utiles pour l'évaluation et la visualisation des différentes méthodes d'optimisation:

In [ ]:
def model(features, weights):
    # features has shape N x K
    # weights has shape K
    # output has shape N
    logits = features@weights
    y_pred_prob = 1 / (1 + np.exp(-logits))

    return (y_pred_prob >= 0.5).astype(int)


def compose_title(title, subtitle=None): # Crée un titre complet pour les figures
    subtitle = "" if subtitle is None else f" ({subtitle})"
    return f"{title}{subtitle}"

def disp_accuracy(weights, features, true_labels, subtitle=None): # Evalue la précision (accuracy) du modèle
    result = model(features, weights)
    errors = np.count_nonzero(result != true_labels) #compare les prédictions de model() aux vraies étiquettes
    accuracy = 1 - errors / true_labels.size

    # Précision conditionnelle : blessés (1) et non blessés (0)
    mask_pos = (true_labels == 1)
    mask_neg = (true_labels == 0)

    # Nombre de vrais positifs / vrais négatifs
    pred1 = np.count_nonzero((result == 1) & mask_pos) / np.count_nonzero(mask_pos)
    pred0 = np.count_nonzero((result == 0) & mask_neg) / np.count_nonzero(mask_neg)

    plt.figure(figsize=(15, 5))
    plt.plot(true_labels, '*', label="Vraie valeur")
    plt.plot(result, '+', label="Valeur prédite")
    plt.xlabel("Athlète")
    plt.ylabel("Classe")
    plt.legend()
    plt.title(compose_title(f"Accuracy of {100*accuracy:.1f}% with {errors} errors", subtitle))
    plt.show() # Affiche un graphique comparant les vraies classes et les prédictions
    # Affichage textuel
    print(f"Accuracy totale : {100*accuracy:.2f}%")
    print(f"Accuracy sur les non-blessés (classe 0) : {100*pred0:.2f}%")
    print(f"Accuracy sur les blessés (classe 1) : {100*pred1:.2f}%")

def disp_weight_variation(poids, subtitle=None): # Montre comment les poids du modèle changent au fil des itérations
    displacement = np.linalg.norm(np.diff(poids, axis=0), axis=1) # Calcule la norme de la différence entre deux itérations successives
    plt.figure(figsize=(15, 5))
    plt.semilogy(displacement) # Trace les variations en échelle logarithmique
    plt.xlabel("Iteration")
    plt.ylabel("Variation in the weight")
    plt.grid()
    plt.title(compose_title("Weight variation", subtitle))
    plt.show()

def disp_losses(losses, subtitle=None): # Trace les courbes de perte sur les itérations, en échelle log
    plt.figure(figsize=(15, 5))
    for name, loss in losses.items(): # dictionnaire permettant de comparer plusieurs méthodes sur le même graphique
        plt.semilogy(loss, label=name)
    plt.xlabel("Iteration")
    plt.grid()
    plt.legend()
    plt.title(compose_title("Loss", subtitle))
    plt.show()

def disp_weight(x, sparse_tol=1e-4, subtitle=None): # visualiser les poids non nuls vs proches de zéro et calcul de sparsité
    plt.figure(figsize=(15, 5))
    i = np.arange(len(x))
    nonzeros = np.abs(x) > sparse_tol
    sparsity = 1 - np.count_nonzero(nonzeros) / x.size
    plt.plot(i[nonzeros], x[nonzeros], '.r', label="non zeros")
    plt.plot(i[~nonzeros], x[~nonzeros], '.g', label=f"zeros (≤ {sparse_tol})")
    plt.xlabel("Feature")
    plt.ylabel("Weight")
    plt.grid()
    plt.title(compose_title(f"Weight matrix with sparsity of {100*sparsity:.1f}%", subtitle))
    plt.legend()
    plt.show()

# Fonction globale qui appelle toute les précédentes
def disp_all_infos(current_method, theta_n, current_loss, lambd, sparse_tol=1e-4, other_losses=dict()):
    print("Method:", current_method) # Affiche nom de la méthode
    print("Iterations:", current_loss.size - 1) # Affiche nombre d'itérations
    print("Last loss:", current_loss[-1]) # Affiche dernière valeur de la perte
    theta_star = theta_n[-1]
    disp_losses(other_losses | {current_method: current_loss}) # Affiche courbe de perte
    disp_weight_variation(theta_n) # Affiche variation des poids
    disp_weight(theta_star, sparse_tol=sparse_tol, subtitle=f"lambda = {lambd}") # Affiche vecteur final des poids
    disp_accuracy(theta_star, x_training_int, y_training, subtitle="training") # Affiche accuracy sur les données d'entraînement
    disp_accuracy(theta_star, x_validation_int, y_validation, subtitle="validation") # Affiche accuracy sur les données de test

# Approche 1 : FISTA

In [ ]:
def algoFISTA(theta0, w0, w1, x, y, max_iter, gamma, lambd, tol=1e-5, a=4):
    """ Algorithme FISTA

    Paramètres
    ----------
    theta0: numpy.ndarray
        Point de départ pour l'itération
    x: numpy.ndarray
        Notre échantillon de données d'entraînement, de taille (N, K)
    y: numpy.ndarray
        Variable binaire indiquant présence/absence de blessure, de taille (N,)
    gamma: float
        Pas de descente de l'algo FISTA step
    max_iter: int
        Nb maximal d'itérations
    lambd: float
        Paramètre de régularization
    tol: float
        Tolérance (critère d'arrêt)
    a: float
        Paramètre de l'lagorithme FISTA

    Returns
    -------
    theta_n: numpy.ndarray
        Séquence de theta_n, de taille (nombre d'itérations + 1, K)
    """
    tk = lambda k: (k + a - 1) / a
    alphak = lambda k: (tk(k) - 1) / tk(k + 1)

    theta_n = [theta0]
    v = theta0

    for k in tqdm(range(max_iter)):
        g = grad_h(v, x, y, w0, w1)
        theta_n.append(prox_f(v - gamma * g, lambd * gamma))
        v = theta_n[-1] + alphak(k) * (theta_n[-1] - theta_n[-2])

        if np.linalg.norm(theta_n[-1] - theta_n[-2]) <= tol:
            break

    return np.array(theta_n)

##1er essai avec l'algo FISTA



In [ ]:
# Paramètres
gamma = 1.9/beta_b
lambd = 1
sparse_tol = 1e-2 # Tolerance when estimating sparsity of the weight vector

N = x_training_int.shape[0]
p = x_training_int.shape[1]

theta0 = np.zeros(p)

theta_n_fista_scaled = algoFISTA(theta0, w0=0.625, w1=2.5, x=x_training_int, y=y_training, max_iter=10_000, gamma=gamma, lambd=lambd)
losses_fista = np.array([h(theta, x_training_int, y_training, 0.625, 2.5) + f(theta, lambd) for theta in theta_n_fista_scaled])

disp_all_infos(
    "FISTA",
    theta_n_fista_scaled,
    losses_fista,
    lambd,
    sparse_tol,
    other_losses = {},
)

# Approche 2 : Chambolle - Pock
voir réf: https://en.wikipedia.org/wiki/Chambolle%E2%80%93Pock_algorithm
(à confirmer)

Dans notre cas, on a $K = Id$, $G = h$ et $F = \lambda \| .\|_1$ (Nous avons pris par défaut $\theta = 1$ car il y a une garanti de convergence dans ce cas là, en s'assurant quand même que $\tau\sigma\leq 1$)

En gardant les mêmes fonctions $f$ et $h$ qu'auparavant, l'algorithme de Chambolle-Pock nécessite de connaître les fonctions $prox_{\sigma f^*}$ et $prox_{\tau h}$  pour $\sigma$ et $\tau$ des pas de descente constants. Nous n'avons pas de forme explicite pour le prox de $h$, toutefois comme nous avons vu que cette fonction est de gradient $\beta$-lipschitz nous allons faire une mise à jour primale sur $\nabla h$. Pour commencer nous definissons donc la fonction $prox_{\sigma f^*}$:


In [ ]:
def prox_fstar(x, lambd):
  return np.clip(x, -lambd, lambd) #projection sur la boule infinie


Quelles valeurs pour $\tau$ et $\sigma$ ? $\tau$ doit peut-être être posé en fonction de $\beta$, par exemple le pas théorique optimal de la descente de gradient à savoir $\frac{1}{\beta}$? Et ensuite on choisit $\sigma = \frac{1}{\tau}$, comme ça on a bien $\tau \sigma \leq 1$ ?


In [ ]:
def algoCP(x, y, theta0, theta_0, sigma, tau, max_iter, lambd, w0, w1, tol=1e-5):
  """ Algorithme Chambolle-Pock

  Paramètres
  ----------
  theta0: numpy.ndarray
      Point de départ pour l'itération
  theta_0: numpy.ndarray
      Point de départ pour l'itération
  x: numpy.ndarray
      Notre échantillon de données d'entraînement, de taille (N, K)
  y: numpy.ndarray
      Variable binaire indiquant présence/absence de blessure, de taille (N,)
  sigma, tau: float
      Pas de descentes de l'algo Chambolle-Pock sur le pb primal et dual
  max_iter: int
      Nb maximal d'itérations
  lambd: float
      Paramètre de régularization
  w0, w1: floats
      Poids permettant d'équilibrer le problème dans la fonction grad_h
  tol: float
      Tolérance (critère d'arrêt)

  Returns
  -------
  theta_n: numpy.ndarray
      Séquence de theta_n, de taille (nombre d'itérations + 1, K)
  """
  #Initialisation
  theta_n = [theta0] # x_n (j'écris ça pour l'instant pour m'y retrouver, on pourra supprimer après)
  thetahat_n = [theta0] # x_bar_n
  thetad_n = [theta_0] #y_n

  for k in tqdm(range(max_iter)):
    thetad_n.append(prox_fstar(thetad_n[-1] + sigma*thetahat_n[-1], lambd))
    theta_n.append(theta_n[-1] - tau*(grad_h(theta_n[-1], x, y, w0, w1) + thetad_n[-1]))
    thetahat_n.append(2*theta_n[-1] - theta_n[-2])

    if np.linalg.norm(theta_n[-1] - theta_n[-2]) <= tol:
            break
  return np.array(theta_n)

In [ ]:
# Paramètres
tau = 1/beta_b
sigma = 0.9/tau
lambd = 1
sparse_tol = 1e-2 # Tolerance when estimating sparsity of the weight vector

N = x_training_int.shape[0]
p = x_training_int.shape[1]

theta0 = np.zeros(p)
theta_0 = np.zeros(p)

theta_n_cp = algoCP(x=x_training_int, y=y_training, theta0 = theta0, theta_0 = theta_0, sigma = sigma, tau = tau, max_iter = 100_000, lambd=lambd, w0=0.625, w1=2.5, tol=1e-5)

losses_cp = np.array([h(theta, x_training_int, y_training, 0.625, 2.5) + f(theta, lambd) for theta in theta_n_cp])

disp_all_infos(
    "Chambolle-Pock",
    theta_n_cp,
    losses_cp,
    lambd,
    sparse_tol,
    other_losses = {},
)

# Approche 3 : Adam

Est-ce que pour l'approche Adam on doit garder le terme de régularization ?